In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from geopy.distance import geodesic 
from statistics import mean 
import json
from datetime import datetime

In [2]:
sidewalk_segments = pd.read_csv('sidewalk_segment3.csv')
intersections = pd.read_excel('intersections_metadata.xlsx')

pano_road_bridge = pd.read_excel('pano_road_bridge.xlsx')
pano_road_bridge = pano_road_bridge.drop_duplicates()

right_json = gpd.read_file("right_sidewalk.geojson")

In [3]:
intersections.shape

(10535, 3)

In [4]:
pano_road_bridge.shape

(33517, 3)

In [5]:
sidewalk_segments.shape

(31213, 12)

In [6]:
pano_road_bridge.head()

,OBJECTID,FID_OutputLines,FID_centerlines_dissolve
0,1,1,6368
1,2,61,14855
2,3,62,17034
3,4,63,17034
4,5,66,15610


In [7]:
pano_road_bridge.columns = ['index', 'intersection_OBJECTID', 'roadsegment_OBJECTID']

In [8]:
sidewalk_segments = sidewalk_segments.filter(['segmentId', 'directionInd', 'linearId', 'streetName', 'startLat', 'startLong', 'endLat', 'endLong', 'whichArcgisFile', 'roadGrade', 'geoJson'])

In [9]:
pano_road_bridge.head()

,index,intersection_OBJECTID,roadsegment_OBJECTID
0,1,1,6368
1,2,61,14855
2,3,62,17034
3,4,63,17034
4,5,66,15610


In [10]:
intersections.head()

,OBJECTID,Long,Lat
0,1,-87.938329,42.957537
1,2,-87.937779,42.955233
2,61,-87.971437,43.026332
3,62,-87.974339,43.047036
4,63,-87.974563,43.046544


In [11]:
pano_road_bridge = pano_road_bridge.merge(intersections, left_on='intersection_OBJECTID', right_on='OBJECTID')

In [12]:
# makes sense - same number of rows as before, now just with more info
pano_road_bridge.shape

(33517, 6)

In [13]:
pano_road_bridge = pano_road_bridge.filter(['intersection_OBJECTID', 'roadsegment_OBJECTID', 'Long', 'Lat'])

In [14]:
pano_road_bridge.head()

,intersection_OBJECTID,roadsegment_OBJECTID,Long,Lat
0,1,6368,-87.938329,42.957537
1,61,14855,-87.971437,43.026332
2,62,17034,-87.974339,43.047036
3,63,17034,-87.974563,43.046544
4,66,15610,-87.968430,43.033238


In [15]:
sidewalk_segments.shape

(31213, 11)

In [16]:
sidewalk_segments.head()

,segmentId,directionInd,linearId,streetName,startLat,startLong,endLat,endLong,whichArcgisFile,roadGrade,geoJson
0,0,E,3371,S 19th St,42.921256,-87.939263,42.920893,-87.939275,right,0.013539,"{""id"": 0, ""type"": ""Feature"", ""properties"": {},..."
1,1,N,3379,W Aspen Dr,42.921360,-87.936933,42.921376,-87.937892,right,0.013539,"{""id"": 1, ""type"": ""Feature"", ""properties"": {},..."
2,2,N,3381,W Aspen Dr,42.921379,-87.938105,42.921395,-87.939098,right,0.016706,"{""id"": 2, ""type"": ""Feature"", ""properties"": {},..."
3,3,E,3404,S 13th St,42.922578,-87.930591,42.920652,-87.930658,right,0.016706,"{""id"": 3, ""type"": ""Feature"", ""properties"": {},..."
4,4,E,3413,S 20th St,42.923533,-87.940446,42.920218,-87.940578,right,0.009984,"{""id"": 4, ""type"": ""Feature"", ""properties"": {},..."


In [17]:
# make table of every pano/road segment/sidewalk segment combo
pano_road_bridge = pano_road_bridge.merge(sidewalk_segments, left_on='roadsegment_OBJECTID', right_on='linearId')

In [18]:
pano_road_bridge.shape

(60933, 15)

In [19]:
pano_road_bridge.tail(20)

,intersection_OBJECTID,roadsegment_OBJECTID,Long,Lat,segmentId,directionInd,linearId,streetName,startLat,startLong,endLat,endLong,whichArcgisFile,roadGrade,geoJson
60913,18984,23592,-87.928346,43.089411,11303,S,23592,State Hwy 57,43.089354,-87.928241,43.089329,-87.927204,right,0.006062,"{""id"": 11303, ""type"": ""Feature"", ""properties"":..."
60914,18984,23592,-87.928346,43.089411,26923,N,23592,State Hwy 57,43.089492,-87.928276,43.089466,-87.927159,left,0.006062,"{""id"": 26923, ""type"": ""Feature"", ""properties"":..."
60915,18990,23592,-87.927095,43.089382,11303,S,23592,State Hwy 57,43.089354,-87.928241,43.089329,-87.927204,right,0.006062,"{""id"": 11303, ""type"": ""Feature"", ""properties"":..."
60916,18990,23592,-87.927095,43.089382,26923,N,23592,State Hwy 57,43.089492,-87.928276,43.089466,-87.927159,left,0.006062,"{""id"": 26923, ""type"": ""Feature"", ""properties"":..."
60917,18984,23593,-87.928346,43.089411,11304,S,23593,State Hwy 57,43.089380,-87.929471,43.089358,-87.928455,right,0.018645,"{""id"": 11304, ""type"": ""Feature"", ""properties"":..."
60918,18984,23593,-87.928346,43.089411,26924,N,23593,State Hwy 57,43.089518,-87.929504,43.089495,-87.928410,left,0.018645,"{""id"": 26924, ""type"": ""Feature"", ""properties"":..."
60919,18985,23593,-87.929575,43.089437,11304,S,23593,State Hwy 57,43.089380,-87.929471,43.089358,-87.928455,right,0.018645,"{""id"": 11304, ""type"": ""Feature"", ""properties"":..."
60920,18985,23593,-87.929575,43.089437,26924,N,23593,State Hwy 57,43.089518,-87.929504,43.089495,-87.928410,left,0.018645,"{""id"": 26924, ""type"": ""Feature"", ""properties"":..."
60921,18984,23594,-87.928346,43.089411,11305,N,23594,W Capitol Dr,43.089468,-87.928451,43.089489,-87.929466,right,0.018645,"{""id"": 11305, ""type"": ""Feature"", ""properties"":..."
60922,18984,23594,-87.928346,43.089411,26925,S,23594,W Capitol Dr,43.089330,-87.928416,43.089353,-87.929512,left,0.018645,"{""id"": 26925, ""type"": ""Feature"", ""properties"":..."


In [20]:
def get_intersect_distance_to_start_and_end(row):
    intersection = (row['Lat'], row['Long'])
    startPoint = (row['startLat'], row['startLong'])
    endPoint = (row['endLat'], row['endLong'])
    intersection_to_start = geodesic(intersection, startPoint)
    intersection_to_end = geodesic(intersection, endPoint)
    return intersection_to_start, intersection_to_end

In [21]:
# NE: 45
# SE: 135
# SW: 225
# NW: 315
def determine_relevant_pano(row):
    pano_degree = 0
    endpoint_to_use = 'start'
    
    intersection_to_start, intersection_to_end = get_intersect_distance_to_start_and_end(row)
    if intersection_to_start > intersection_to_end:
        # then intersection to end is closer - change endpoint to use
        endpoint_to_use = 'end'
    
    if row['directionInd'] in ['N', 'S']:
        # the intersection is on left or right side of street based on LONGITUDE
        # so we want to see whether this intersection LONG is to the west or east of the street
        if row['Long'] < row[endpoint_to_use + 'Long']:
            # intersection has smaller longitude, so it is to the WEST of the sidewalk segment
            # we want the pano facing NE or SE
            if row['directionInd'] == 'N':
                pano_degree = 45
            else:
                pano_degree = 135
        else:
            # intersection has larger longitude, so it is to the EAST of the sidewalk segment
            # we want the pano facing NW or SW
            if row['directionInd'] == 'N':
                pano_degree = 315
            else:
                pano_degree = 225
            
    elif row['directionInd'] in ['E', 'W']:
        # the intersection is on the top or bottom of the street based on LATITUDE
        # so we want to see whether this intersection LAT is north or south of the street
        if row['Lat'] < row[endpoint_to_use + 'Lat']:

            # intersection has smaller latitude, so it is SOUTH of the sidewalk segment
            # we want the pano facing NW or NE
            if row['directionInd'] == 'E':
                pano_degree = 45
            else:
                pano_degree = 315
        
        else:                
            # intersection has larger latitude, so it is NORTH of the sidewalk segment
            # we want the pano facing SW or SE  
            if row['directionInd'] == 'E':
                pano_degree = 135
            else:
                pano_degree = 225
    
    return (pano_degree, endpoint_to_use)

In [22]:
pano_road_bridge['heading_and_endpoint'] = pano_road_bridge.apply(determine_relevant_pano, axis=1)

In [23]:
def extract_heading(row):
    return int(row['heading_and_endpoint'][0])

def extract_endpoint(row):
    return row['heading_and_endpoint'][1]

pano_road_bridge['heading_direction'] = pano_road_bridge.apply(extract_heading, axis=1)
pano_road_bridge['endpoint'] = pano_road_bridge.apply(extract_endpoint, axis=1)

In [24]:
pano_road_bridge.shape

(60933, 18)

In [36]:
df_to_load = pd.DataFrame()
df_to_load['linkId'] = [i+1 for i in pano_road_bridge.index.tolist()] 
df_to_load['segmentId'] = pano_road_bridge['segmentId']
df_to_load['whichEndpoint'] = pano_road_bridge['endpoint']
df_to_load['panoId'] = pano_road_bridge['intersection_OBJECTID']
df_to_load['headingDegree'] = pano_road_bridge['heading_direction']

In [37]:
df_to_load = df_to_load[18849:]

In [38]:
df_to_load.head()

,linkId,segmentId,whichEndpoint,panoId,headingDegree
18849,18850,29216,start,7413,45
18850,18851,13586,end,18069,225
18851,18852,29216,end,18069,135
18852,18853,13352,start,7414,315
18853,18854,28982,start,7414,225


In [39]:
import pymysql
connection = pymysql.connect(host='aa6f8tcntjfd0z.c5625ddefrth.us-west-2.rds.amazonaws.com',
                             port=3306,
                             user=# ask Emily :) ,
                             password= # ask Emily  ,
                             db='ebdb')
cursor=connection.cursor()

In [40]:
# NOTE: this method is way slow
# creating column list for insertion
cols = "`,`".join([str(i) for i in df_to_load.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in df_to_load.iterrows():
    sql = "INSERT INTO `segment_to_pano3` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()